In [1]:
import json
from tqdm import tqdm
from glob import glob

files = glob('texts-theblackcat102.jsonl*.splitted.requested')
files

['texts-theblackcat102.jsonl14.splitted.requested',
 'texts-theblackcat102.jsonl07.splitted.requested',
 'texts-theblackcat102.jsonl06.splitted.requested',
 'texts-theblackcat102.jsonl04.splitted.requested',
 'texts-theblackcat102.jsonl05.splitted.requested',
 'texts-theblackcat102.jsonl09.splitted.requested',
 'texts-theblackcat102.jsonl08.splitted.requested',
 'texts-theblackcat102.jsonl11.splitted.requested',
 'texts-theblackcat102.jsonl13.splitted.requested',
 'texts-theblackcat102.jsonl03.splitted.requested',
 'texts-theblackcat102.jsonl00.splitted.requested',
 'texts-theblackcat102.jsonl12.splitted.requested',
 'texts-theblackcat102.jsonl10.splitted.requested',
 'texts-theblackcat102.jsonl01.splitted.requested',
 'texts-theblackcat102.jsonl02.splitted.requested']

In [2]:
mapping = {}
for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            if 'Source text\nclear\nLook up details' in data['r']['result']:
                continue
            if 'clear\nLook up details' in data['r']['result']:
                continue
            if data['r']['result'].startswith('Source text\n'):
                continue
                
            mapping[data['src']] = data['r']['result']

6660it [00:00, 158673.00it/s]
49552it [00:00, 159634.76it/s]
49495it [00:00, 168219.98it/s]
49453it [00:00, 174874.50it/s]
49334it [00:00, 172525.69it/s]
49492it [00:00, 168257.91it/s]
49415it [00:00, 172263.46it/s]
49419it [00:00, 168491.01it/s]
49488it [00:00, 169921.45it/s]
49328it [00:00, 163248.93it/s]
49467it [00:00, 170768.93it/s]
49545it [00:00, 177882.81it/s]
49474it [00:00, 172191.35it/s]
49464it [00:00, 165565.03it/s]
49500it [00:00, 172736.07it/s]


In [3]:
len(mapping)

505997

In [4]:
data = []
with open('train.jsonl') as fopen:
    for l in fopen:
        data.append(json.loads(l))

In [5]:
len(data)

50496

In [6]:
for i in range(len(data)):
    for k in range(len(data[i]['conversations'])):
        ms = mapping.get(data[i]['conversations'][k]['text'])
        data[i]['conversations'][k]['text_ms'] = ms

In [7]:
with open('gather-theblackcat102.jsonl', 'w') as fopen:
    for d in data:
        fopen.write(f'{json.dumps(d)}\n')

In [8]:
import random

In [9]:
break_at = [
    'help.openai.com',
]

In [10]:
all_texts = []
for i in tqdm(range(len(data))):
    texts = []
    for c in data[i]['conversations']:
        if c['user'] == 'human':
            t = '<manusia>: '
            n = c['text_ms'] if random.random() < 0.6 and c['text_ms'] else c['text']
            n = n.strip()
            if len(n) < 3:
                break
            t += n
        else:
            if c['text_ms'] is None:
                break
            if any([b in c['text_ms'].lower() for b in break_at]):
                # print(c['text_ms'], '\n-----\n')
                break
            n = c['text_ms']
            n = n.strip()
            if len(n) < 3:
                break
            t = '<bot>: ' + n
        texts.append(t)
    if len(texts) % 2 != 0:
        texts = texts[:-1]
    if not len(texts):
        continue
    while len(texts) and texts[-1].startswith('<manusia>'):
        texts = texts[:-1]
    if not len(texts):
        continue
    if not texts[0].startswith('<manusia>'):
        continue
    all_texts.append('\n'.join(texts))

100%|██████████████████████████████████| 50496/50496 [00:00<00:00, 68424.47it/s]


In [11]:
len(all_texts)

41530

In [12]:
with open('theblackcat102-all_texts.jsonl', 'w') as fopen:
    for t in all_texts:
        fopen.write(f'{json.dumps(t)}\n')

In [13]:
with open('theblackcat102-all_texts.jsonl') as fopen:
    for l in fopen:
        if 'Source text' in l:
            print(l)